# Predicting Columns in a Table - Deployment Optimization

This tutorial will cover how to perform the end-to-end AutoML process to create an optimized and deployable
AutoGluon artifact for production usage.

This tutorial assumes you have already read Predicting Columns in a Table - Quick Start and Predicting Columns in a Table - In Depth

## Fitting a TabularPredictor

We will again use the AdultIncome dataset as in the previous tutorials and train a predictor
to predict whether the person's income exceeds $50,000 or not, which is recorded in the `class` column of this table.

In [ ]:
!pip install autogluon.tabular[all]


INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 k

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
label = 'class'
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


In [ ]:
save_path = 'agModels-predictClass-deployment'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       11.40 GB / 12.67 GB (90.0%)
Disk Space Avail:   62.67 GB / 107.72 GB (58.2%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in 

Next, load separate test data to demonstrate how to make predictions on new examples at inference time:

In [ ]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States,<=50K
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States,<=50K
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States,>50K
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States,<=50K
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States,<=50K


We use our trained models to make predictions on the new data:

In [ ]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data)
y_pred

,class
0,<=50K
1,<=50K
2,>50K
3,<=50K
4,<=50K
...,...
9764,<=50K
9765,<=50K
9766,<=50K
9767,<=50K


We can use leaderboard to evaluate the performance of each individual trained model on our labeled test data:

In [ ]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842870,0.84,accuracy,0.220764,0.101419,0.973490,0.220764,0.101419,0.973490,1,True,3
1,CatBoost,0.842461,0.85,accuracy,0.011849,0.004868,2.912053,0.011849,0.004868,2.912053,1,True,5
2,RandomForestEntr,0.841130,0.83,accuracy,0.240617,0.079518,1.117594,0.240617,0.079518,1.117594,1,True,4
3,XGBoost,0.839902,0.85,accuracy,0.058575,0.009126,0.333443,0.058575,0.009126,0.333443,1,True,9
4,LightGBM,0.839799,0.85,accuracy,0.056704,0.005708,0.285561,0.056704,0.005708,0.285561,1,True,2
5,WeightedEnsemble_L2,0.839799,0.85,accuracy,0.058679,0.006504,0.353218,0.001975,0.000796,0.067657,2,True,12
6,NeuralNetTorch,0.837138,0.83,accuracy,0.071411,0.021648,7.419735,0.071411,0.021648,7.419735,1,True,10
7,LightGBMXT,0.836421,0.83,accuracy,0.018863,0.003975,14.903080,0.018863,0.003975,14.903080,1,True,1
8,ExtraTreesGini,0.833862,0.82,accuracy,0.249596,0.089842,0.780182,0.249596,0.089842,0.780182,1,True,6
9,ExtraTreesEntr,0.833862,0.81,accuracy,0.252799,0.091373,0.791780,0.252799,0.091373,0.791780,1,True,7


## Snapshot a Predictor with .clone()

Now that we have a working predictor artifact, we may want to alter it in a variety of ways to better suite our needs.
For example, we may want to delete certain models to reduce disk usage via `.delete_models()`,
or train additional models on top of the ones we already have via `.fit_extra()`.

While you can do all of these operations on your predictor,
you may want to be able to be able to revert to a prior state of the predictor in case something goes wrong.
This is where `predictor.clone()` comes in.

`predictor.clone()` allows you to create a snapshot of the given predictor,
cloning the artifacts of the predictor to a new location.
You can then freely play around with the predictor and always load
the earlier snapshot in case you want to undo your actions.

All you need to do to clone a predictor is specify a new directory path to clone to:

In [ ]:
save_path_clone = save_path + '-clone'
# will return the path to the cloned predictor, identical to save_path_clone
path_clone = predictor.clone(path=save_path_clone)

Cloned TabularPredictor located in '/content/agModels-predictClass-deployment' to 'agModels-predictClass-deployment-clone'.
	To load the cloned predictor: predictor_clone = TabularPredictor.load(path="agModels-predictClass-deployment-clone")


Note that this logic doubles disk usage, as it completely clones
every predictor artifact on disk to make an exact replica.

Now we can load the cloned predictor:

In [ ]:
predictor_clone = TabularPredictor.load(path=path_clone)
# You can alternatively load the cloned TabularPredictor at the time of cloning:
# predictor_clone = predictor.clone(path=save_path_clone, return_clone=True)

We can see that the cloned predictor has the same leaderboard and functionality as the original:

In [ ]:
y_pred_clone = predictor.predict(test_data)
y_pred_clone

,class
0,<=50K
1,<=50K
2,>50K
3,<=50K
4,<=50K
...,...
9764,<=50K
9765,<=50K
9766,<=50K
9767,<=50K


In [ ]:
y_pred.equals(y_pred_clone)

True

In [ ]:
predictor_clone.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842870,0.84,accuracy,0.218841,0.101419,0.973490,0.218841,0.101419,0.973490,1,True,3
1,CatBoost,0.842461,0.85,accuracy,0.010205,0.004868,2.912053,0.010205,0.004868,2.912053,1,True,5
2,RandomForestEntr,0.841130,0.83,accuracy,0.231455,0.079518,1.117594,0.231455,0.079518,1.117594,1,True,4
3,XGBoost,0.839902,0.85,accuracy,0.063457,0.009126,0.333443,0.063457,0.009126,0.333443,1,True,9
4,LightGBM,0.839799,0.85,accuracy,0.070273,0.005708,0.285561,0.070273,0.005708,0.285561,1,True,2
5,WeightedEnsemble_L2,0.839799,0.85,accuracy,0.072167,0.006504,0.353218,0.001894,0.000796,0.067657,2,True,12
6,NeuralNetTorch,0.837138,0.83,accuracy,0.071476,0.021648,7.419735,0.071476,0.021648,7.419735,1,True,10
7,LightGBMXT,0.836421,0.83,accuracy,0.018437,0.003975,14.903080,0.018437,0.003975,14.903080,1,True,1
8,ExtraTreesEntr,0.833862,0.81,accuracy,0.240686,0.091373,0.791780,0.240686,0.091373,0.791780,1,True,7
9,ExtraTreesGini,0.833862,0.82,accuracy,0.249807,0.089842,0.780182,0.249807,0.089842,0.780182,1,True,6


Now let's do some extra logic with the clone, such as calling refit_full:

In [ ]:
predictor_clone.refit_full()

predictor_clone.leaderboard(test_data)

Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBMXT_FULL ...
	Fitting with cpus=1, gpus=0, mem=0.0/11.0 GB
	0.26s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBM_FULL ...
	Fitting with cpus=1, gpus=0, mem=0.0/11.0 GB
	0.25s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: RandomForestGini_FULL ...
	Fitting with cpus=2, gpus=0
	0.79s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: RandomForestEntr_FULL ...
	Fitting with cpus=2

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_FULL,0.842870,NaN,accuracy,0.008210,NaN,0.035266,0.008210,NaN,0.035266,1,True,17
1,RandomForestGini,0.842870,0.84,accuracy,0.392435,0.101419,0.973490,0.392435,0.101419,0.973490,1,True,3
2,CatBoost,0.842461,0.85,accuracy,0.017748,0.004868,2.912053,0.017748,0.004868,2.912053,1,True,5
3,RandomForestEntr,0.841130,0.83,accuracy,0.390665,0.079518,1.117594,0.390665,0.079518,1.117594,1,True,4
4,LightGBM_FULL,0.840823,NaN,accuracy,0.071326,NaN,0.253297,0.071326,NaN,0.253297,1,True,14
5,WeightedEnsemble_L2_FULL,0.840823,NaN,accuracy,0.073331,NaN,0.320954,0.002006,NaN,0.067657,2,True,24
6,XGBoost,0.839902,0.85,accuracy,0.061799,0.009126,0.333443,0.061799,0.009126,0.333443,1,True,9
7,LightGBM,0.839799,0.85,accuracy,0.089638,0.005708,0.285561,0.089638,0.005708,0.285561,1,True,2
8,WeightedEnsemble_L2,0.839799,0.85,accuracy,0.091583,0.006504,0.353218,0.001945,0.000796,0.067657,2,True,12
9,RandomForestGini_FULL,0.839390,NaN,accuracy,0.219369,NaN,0.785457,0.219369,NaN,0.785457,1,True,15


We can see that we were able to fit additional models, but for whatever reason we may want to undo this operation.

Luckily, our original predictor is untouched!

In [ ]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842870,0.84,accuracy,0.224160,0.101419,0.973490,0.224160,0.101419,0.973490,1,True,3
1,CatBoost,0.842461,0.85,accuracy,0.010386,0.004868,2.912053,0.010386,0.004868,2.912053,1,True,5
2,RandomForestEntr,0.841130,0.83,accuracy,0.220474,0.079518,1.117594,0.220474,0.079518,1.117594,1,True,4
3,XGBoost,0.839902,0.85,accuracy,0.058037,0.009126,0.333443,0.058037,0.009126,0.333443,1,True,9
4,LightGBM,0.839799,0.85,accuracy,0.060846,0.005708,0.285561,0.060846,0.005708,0.285561,1,True,2
5,WeightedEnsemble_L2,0.839799,0.85,accuracy,0.062802,0.006504,0.353218,0.001956,0.000796,0.067657,2,True,12
6,NeuralNetTorch,0.837138,0.83,accuracy,0.069706,0.021648,7.419735,0.069706,0.021648,7.419735,1,True,10
7,LightGBMXT,0.836421,0.83,accuracy,0.022560,0.003975,14.903080,0.022560,0.003975,14.903080,1,True,1
8,ExtraTreesGini,0.833862,0.82,accuracy,0.241432,0.089842,0.780182,0.241432,0.089842,0.780182,1,True,6
9,ExtraTreesEntr,0.833862,0.81,accuracy,0.263367,0.091373,0.791780,0.263367,0.091373,0.791780,1,True,7


We can simply clone a new predictor from our original, and we will no longer be impacted
by the call to refit_full on the prior clone.

## Snapshot a deployment optimized Predictor via .clone_for_deployment()

Instead of cloning an exact copy, we can instead clone a copy
which has the minimal set of artifacts needed to do prediction.

Note that this optimized clone will have very limited functionality outside of calling predict and predict_proba.
For example, it will be unable to train more models.

In [ ]:
save_path_clone_opt = save_path + '-clone-opt'
# will return the path to the cloned predictor, identical to save_path_clone_opt
path_clone_opt = predictor.clone_for_deployment(path=save_path_clone_opt)

Cloned TabularPredictor located in '/content/agModels-predictClass-deployment' to 'agModels-predictClass-deployment-clone-opt'.
	To load the cloned predictor: predictor_clone = TabularPredictor.load(path="agModels-predictClass-deployment-clone-opt")
Clone: Keeping minimum set of models required to predict with best model 'WeightedEnsemble_L2'...
Deleting model LightGBMXT. All files under /content/agModels-predictClass-deployment-clone-opt/models/LightGBMXT will be removed.
Deleting model RandomForestGini. All files under /content/agModels-predictClass-deployment-clone-opt/models/RandomForestGini will be removed.
Deleting model RandomForestEntr. All files under /content/agModels-predictClass-deployment-clone-opt/models/RandomForestEntr will be removed.
Deleting model CatBoost. All files under /content/agModels-predictClass-deployment-clone-opt/models/CatBoost will be removed.
Deleting model ExtraTreesGini. All files under /content/agModels-predictClass-deployment-clone-opt/models/ExtraT

In [ ]:
predictor_clone_opt = TabularPredictor.load(path=path_clone_opt)

To avoid loading the model in every prediction call, we can persist the model in memory by:

In [ ]:
predictor_clone_opt.persist()

Persisting 2 models in memory. Models will require 0.0% of memory.


['WeightedEnsemble_L2', 'LightGBM']

We can see that the optimized clone still makes the same predictions:

In [ ]:
y_pred_clone_opt = predictor_clone_opt.predict(test_data)
y_pred_clone_opt

,class
0,<=50K
1,<=50K
2,>50K
3,<=50K
4,<=50K
...,...
9764,<=50K
9765,<=50K
9766,<=50K
9767,<=50K


In [ ]:
y_pred.equals(y_pred_clone_opt)

True

In [ ]:
predictor_clone_opt.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.839799,0.85,accuracy,0.156480,0.005708,0.285561,0.156480,0.005708,0.285561,1,True,1
1,WeightedEnsemble_L2,0.839799,0.85,accuracy,0.157477,0.006504,0.353218,0.000997,0.000796,0.067657,2,True,2


We can check the disk usage of the optimized clone compared to the original:

In [ ]:
size_original = predictor.disk_usage()
size_opt = predictor_clone_opt.disk_usage()
print(f'Size Original:  {size_original} bytes')
print(f'Size Optimized: {size_opt} bytes')
print(f'Optimized predictor achieved a {round((1 - (size_opt/size_original)) * 100, 1)}% reduction in disk usage.')

Size Original:  18359276 bytes
Size Optimized: 191538 bytes
Optimized predictor achieved a 99.0% reduction in disk usage.


We can also investigate the difference in the files that exist in the original and optimized predictor.

Original:

In [ ]:
predictor.disk_usage_per_file()

,size
/models/ExtraTreesGini/model.pkl,5065815
/models/ExtraTreesEntr/model.pkl,5024045
/models/RandomForestGini/model.pkl,3408790
/models/RandomForestEntr/model.pkl,3267189
/models/XGBoost/xgb.ubj,506961
/models/LightGBMLarge/model.pkl,310769
/models/NeuralNetTorch/model.pkl,253754
/models/NeuralNetFastAI/model-internals.pkl,170508
/models/LightGBM/model.pkl,147692
/models/CatBoost/model.pkl,52154


Optimized:

In [ ]:
predictor_clone_opt.disk_usage_per_file()

,size
/models/LightGBM/model.pkl,147720
/metadata.json,19712
/learner.pkl,10217
/models/WeightedEnsemble_L2/model.pkl,10154
/models/trainer.pkl,2827
/predictor.pkl,903
/version.txt,5


## Compile models for maximized inference speed

In order to further improve inference efficiency, we can call `.compile()` to automatically
convert sklearn function calls into their ONNX equivalents.
Note that this is currently an experimental feature, which only improves RandomForest and TabularNeuralNetwork models.
The compilation and inference speed acceleration require installation of `skl2onnx` and `onnxruntime` packages.
To install supported versions of these packages automatically, we can call `pip install autogluon.tabular[skl2onnx]`
on top of an existing AutoGluon installation, or `pip install autogluon.tabular[all,skl2onnx]` on a new AutoGluon installation.

It is important to make sure the predictor is cloned, because once the models are compiled, it won't support fitting.

In [ ]:
predictor_clone_opt.compile()

Compiling 2 Models ...
Skipping compilation for WeightedEnsemble_L2 ... (No config specified)
Skipping compilation for LightGBM ... (No config specified)
Finished compiling models, total runtime = 0s.


With the compiled predictor, the prediction results might not be exactly the same but should be very close.

In [ ]:
y_pred_compile_opt = predictor_clone_opt.predict(test_data)
y_pred_compile_opt

,class
0,<=50K
1,<=50K
2,>50K
3,<=50K
4,<=50K
...,...
9764,<=50K
9765,<=50K
9766,<=50K
9767,<=50K


Now all that is left is to upload the optimized predictor to a centralized storage location such as S3.
To use this predictor in a new machine / system, simply download the artifact to local disk and load the predictor.
Ensure that when loading a predictor you use the same Python version
and AutoGluon version used during training to avoid instability.